<a href="https://colab.research.google.com/github/vmatiasw/modelos_y_simulacion/blob/main/P4E6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [328]:
import pandas as pd
from collections import Counter
import random
import time
import numpy as np
import math

In [329]:
def medir_tiempo(func):
    inicio = time.perf_counter()
    resultado = func()
    fin = time.perf_counter()
    return resultado, fin - inicio

def obtener_DataFrame(dict_fpm, P):
  data = {
      "X": list(dict_fpm.keys()),
      "P(X) Real": [P[x] for x in dict_fpm.keys()],
      "Estimación": list(dict_fpm.values()),
      "Diferencia": [abs(P[x] - dict_fpm[x]) for x in dict_fpm.keys()]
  }
  return pd.DataFrame(data)

def estimar_dict_fpm(p, soporte, n):
  """
  Estima la función de probabilidad de masa de una variable aleatoria discreta utilizando un generador de muestras.
  """
  muestras = [p() for _ in range(n)]
  frecuencias = Counter(muestras)

  dict_fpm = {x: frecuencias.get(x, 0) / n for x in soporte}

  return dict_fpm

def calcular_TVD(dict_fpm_estimada, dict_fpm_real):
  """
  Calcula la Distancia Total de Variación (TVD) entre dos distribuciones discretas.
  """
  return 0.5 * sum(abs(dict_fpm_real[x] - dict_fpm_estimada.get(x, 0)) for x in dict_fpm_real)

def calificar_generador(generador_de_muestras, dict_fpm_correcta):
  n = 100_000
  soporte = dict_fpm_correcta.keys()
  dict_fpm_estimada, tiempo = medir_tiempo(lambda: estimar_dict_fpm(generador_de_muestras, soporte, n))
  print(obtener_DataFrame(dict_fpm_estimada, dict_fpm_correcta))
  tvd = calcular_TVD(dict_fpm_estimada, dict_fpm_correcta)
  print()
  print(f"Distancia total de variación (TVD): {tvd:.6f}")
  print(f"Tiempo de ejecución: {tiempo/n:.6f} segundos")


In [330]:
DICT_FPM = {i+1: p for i, p in enumerate([0.15, 0.20, 0.10, 0.35, 0.20])}
SOPORTE = DICT_FPM.keys()
print('funcion de probabilidad de masa: ',DICT_FPM)
print('debe dar 1: ',sum(DICT_FPM.values()))

funcion de probabilidad de masa:  {1: 0.15, 2: 0.2, 3: 0.1, 4: 0.35, 5: 0.2}
debe dar 1:  1.0


In [331]:
# Transformada inversa.
def ordenar_dict_fpm(dict_fpm):
  return dict(sorted(dict_fpm.items(), key=lambda x: x[1], reverse=True))

def calcular_dict_fda(dict_fpm):
  """
  Recibe una funcion de probabilidad de masa en forma de diccionario de pares
  (valor, probabilidad) y devuelve su funcion de distribucion acumulada.
  Solo definidas para el soporte.
  """
  F = {}
  suma = 0
  for valor, probabilidad in dict_fpm.items():
    suma += probabilidad
    F[valor] = suma
  return F

def TI(dict_fda):
  u = random.random()
  for v, p in F.items():
    if u < p:
      return v

dict_fda = calcular_dict_fda(ordenar_dict_fpm(DICT_FPM))
generador_de_muestras = lambda: TI(dict_fda)
calificar_generador(generador_de_muestras, DICT_FPM)

   X  P(X) Real  Estimación  Diferencia
0  1       0.15     0.11064     0.03936
1  2       0.20     0.13963     0.06037
2  3       0.10     0.18938     0.08938
3  4       0.35     0.05002     0.29998
4  5       0.20     0.12019     0.07981

Distancia total de variación (TVD): 0.284450
Tiempo de ejecución: 0.000001 segundos
